In [1]:
from yaml import safe_load
from plot_results import GetFlatInfoResults, IncludeLists, GetHyperParams, GetFinalVals
import numpy as np
import plotly
import plotly.graph_objs as go


In [10]:
files = []
with open('viz_experiments.txt') as f:
  for l in f.readlines():
    files.append(l.strip())

In [11]:
results = []
for fname in files:
  with open(fname, 'r') as f:
    yaml = safe_load(f.read())
    if not yaml:
      print("empty:", f)
    results.append(yaml)

results = [r for r in results if not 'exit_info' in r]


In [16]:
train_results = {r['exp_info']['experiment_hash']: r['train'] for r in results}

In [17]:
flats = [GetFlatInfoResults(x, files) for x in results]

In [45]:
flats[0].keys()

dict_keys(['train_time', 'accuracy', 'cos_acc', 'cos_loss', 'dot_acc', 'dot_xent_loss', 'global_step', 'mse_loss', 'vmf_simp_loss', 'char_to_idx_file', 'embedding_file', 'experiment_hash', 'experiment_name', 'finished_date', 'max_mem_alloc', 'size_bytes', 'size_params', 'vocab_file', 'word_length', 'add_random_count', 'batch_size', 'char_embedding_size', 'char_vocab_size', 'conv_activation', 'dot_loss_weight', 'end_of_word_loss_weight', 'epochs', 'eval_acc', 'learning_rate', 'learning_rate_cap', 'loss_fn', 'lr_decay', 'lr_step_size', 'misspelling_rate', 'misspelling_transforms', 'mse_loss_weight', 'optimizer', 'random_seed', 'run_validation', 'seg1.kernel_size', 'seg1_type', 'space_freq', 'token_embedding_size', 'hash', 'uid', 'model_size_range_bytes', 'kernel|filter_sizes'])

In [49]:
labels = ['train_time', 'accuracy', 'cos_acc', 'cos_loss', 'dot_acc', 'dot_xent_loss', 'global_step', 'mse_loss', 'vmf_simp_loss', 'char_to_idx_file', 'embedding_file', 'experiment_hash', 'experiment_name', 'finished_date', 'max_mem_alloc', 'size_bytes', 'size_params', 'vocab_file', 'word_length', 'add_random_count', 'batch_size', 'char_embedding_size', 'char_vocab_size', 'conv_activation', 'dot_loss_weight', 'end_of_word_loss_weight', 'epochs', 'eval_acc', 'learning_rate', 'learning_rate_cap', 'loss_fn', 'lr_decay', 'lr_step_size', 'misspelling_rate', 'misspelling_transforms', 'mse_loss_weight', 'optimizer', 'random_seed', 'run_validation', 'seg1.kernel_size', 'seg1_type', 'space_freq', 'token_embedding_size', 'hash', 'uid', 'model_size_range_bytes', 'kernel|filter_sizes']
def ExpStr(flat):
  s = ""
  for k in labels:
    val = flat[k]
    if type(val)==int:
      s+= "%s: %d<br>" % (k, flat[k])
    if type(val)==float:
      s+= "%s: %.7f<br>" % (k, flat[k])
    if type(val)==str:
      s+= "%s: %s<br>" % (k, flat[k])

  return s

In [50]:
def ScatterOverTraining(flat, x, y):
  h = flat['experiment_hash']
  tr = train_results[h]
  xs = tr[x]
  ys = tr[y]
  text = ExpStr(flat)
  return {'type': 'scatter',
          'mode': 'lines',
          'x':xs,
          'y':ys,
          'showlegend':False,
          'text': text,  'textposition':"top left",
          'hoverinfo': 'text',
          'marker': {}}


In [51]:
dat = ScatterOverTraining(flats[0], 'gl_step', 'accuracy')

In [52]:
scatter = go.Figure(data=[dat])
plotly.offline.plot(scatter, filename='/home/tucker/Downloads/tmp.html')


'/home/tucker/Downloads/tmp.html'

In [53]:
dats = []
for f in flats:
  dats.append(ScatterOverTraining(f, 'gl_step', 'accuracy'))

In [54]:
scatter = go.Figure(data=dats)
plotly.offline.plot(scatter, filename='/home/tucker/Downloads/tmp.html')

'/home/tucker/Downloads/tmp.html'

In [23]:
import pandas as pd

In [77]:
df = pd.read_csv('isolate_emb_vs_tok_ish/initial_acc_and_emb_pred_results.csv')

In [78]:
df['cosmse'] = df['cos_loss']/(df['mse_loss']**.01)

In [79]:
corrmat = df.corrwith(df['overall_acc'])

In [80]:
df.sort_values('mse_loss')[['mse_loss', 'overall_acc', 'uid', 'cos_loss']]

,mse_loss,overall_acc,uid,cos_loss
12,0.000883,0.682638,c6ce592a5,-0.949945
5,0.002334,0.432953,10012ffe2,-0.839984
13,0.002352,0.686201,3ad077b9d,-0.952266
1,0.002455,0.545317,82e0468a4,-0.854238
9,0.002501,0.561506,ef800feb9,-0.849569
8,0.014240,0.172716,8822634bc,-0.675679
11,0.015827,0.203669,3683c34d6,-0.691383
2,0.016105,0.251367,0c478f576,-0.705317
3,0.017376,0.317334,95d7b74c4,-0.726190
6,0.017970,0.347873,5023ef12a,-0.738165


In [81]:
corrmat

accuracy         0.541293
cos_acc          0.860676
cos_loss        -0.937799
dot_acc         -0.703254
dot_xent_loss    0.843105
mse_loss        -0.682015
vmf_simp_loss    0.785805
overall_acc      1.000000
cosmse          -0.941235
dtype: float64

In [1]:
import numpy as np

In [49]:
r = (np.random.rand(200000,20)>.5).astype(np.int).astype(str)

In [50]:
idxs = []
for s in r:
  idx = "".join(s).find('11')
  if idx == -1:
    idx = len(s)+1
  idx+=2
  #print("".join(s), idx)
  idxs.append(idx)

In [51]:
np.array(idxs).mean()

5.967395

In [19]:
"".join(r[0])

'01001111000101110111'

In [64]:
n=100000000
sumn = (np.random.normal(0,1, (n))>2).sum()

In [67]:
1/(sumn/n)

43.993501279990916

In [68]:
def reward_to_go(rews):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
    return rtgs

In [69]:
reward_to_go([1,0,1,0,0,0,1,0,0,1,1,1,0,1])


array([7, 6, 6, 5, 5, 5, 5, 4, 4, 4, 3, 2, 1, 1])